## Video Capture and Display in a window

In [43]:
import cv2
import os

def capture_and_display_video(output_folder):
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    cv2.namedWindow("Webcam Video", cv2.WINDOW_NORMAL)

    count=0
    while True:
        
        ret, frame = cap.read()

        
        if not ret:
            print("Error: Failed to capture frame.")
            break
        
        # image_path = os.path.join(output_folder, f"frame_{count:04d}.png")
        # cv2.imwrite(image_path, frame) #uncomment this line to save the frames

        count+=1
       
        cv2.imshow("Webcam Video", frame)
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

   
    cap.release()
    cv2.destroyAllWindows()

output_folder = r'C:\Users\ramak\Videos\captured_frames'
capture_and_display_video(output_folder)


In [45]:
import cv2
import os
def capture_disp(output_folder):
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    cv2.namedWindow("Webcam Video", cv2.WINDOW_NORMAL)
    count=0
    while True:        
        ret, frame = cap.read()
        if not ret:
            print("Error")
            break
        
        # image_path = os.path.join(output_folder, f"frame_{count:04d}.png")
        # cv2.imwrite(image_path, frame)
        count+=1
       
        cv2.imshow("Webcam Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

output_folder = r'C:\Users\ramak\Videos\captured_frames'
capture_disp(output_folder)

## Chroma Keying and creating a composite video of the two videos


In [ ]:
import cv2
import numpy as np

video1 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4")
video2 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4")

width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))

video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (width, height))
kernel = np.ones((3, 3), np.uint8)
while True:    
    ret1, frame1 = video1.read()
    ret2, frame2 = video2.read()
    if not ret1 or not ret2:
        break
    hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    lower_green = np.array([36, 0, 60])
    upper_green = np.array([70, 255, 255])

    mask1 = cv2.inRange(hsv1, lower_green, upper_green)
    mask1 = cv2.erode(mask1, kernel, iterations=1)
    mask_inv = cv2.bitwise_not(mask1)

    non_green_region = cv2.bitwise_and(frame1, frame1, mask=mask_inv)
    green_region = cv2.bitwise_and(frame2, frame2, mask=mask1)
    output_frame = cv2.add(green_region, non_green_region)

    video_writer.write(output_frame)


video1.release()
video2.release()
video_writer.release()


## Other ways of Chroma Keying that was tried - Experimental

In [66]:
import cv2
import numpy as np

def chroma_keying(video_path1, video_path2, output_path, chroma_color=(0, 255, 0), threshold=60):
    # Open the videos
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    # Get video properties
    fps = int(cap1.get(cv2.CAP_PROP_FPS))
    width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        # Resize frame2 to match the dimensions of frame1
        frame2 = cv2.resize(frame2, (width, height))

        # Convert frames to HSV
        hsv_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)

        # Define the chroma key color range
        lower_color = np.array([chroma_color[0] - threshold, chroma_color[1] - threshold, chroma_color[2] - threshold])
        upper_color = np.array([chroma_color[0] + threshold, chroma_color[1] + threshold, chroma_color[2] + threshold])

        # Create a mask using the chroma key color range
        mask = cv2.inRange(hsv_frame1, lower_color, upper_color)

        # Invert the mask
        inverted_mask = cv2.bitwise_not(mask)

        # Apply the mask to the original frame
        foreground = cv2.bitwise_and(frame1, frame1, mask=inverted_mask)

        # Apply the mask to the second video frame
        background = cv2.bitwise_and(frame2, frame2, mask=mask)

        # Combine the foreground and background
        result = cv2.add(foreground, background)

        # Write the frame to the output video
        video_writer.write(result)

        cv2.imshow("Chroma Keying", result)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release video capture and writer objects
    cap1.release()
    cap2.release()
    video_writer.release()
    cv2.destroyAllWindows()

# def chroma_keying(video_path1, video_path2, output_path, chroma_color=(0, 255, 0), threshold=60):
#     # Open the videos
#     cap1 = cv2.VideoCapture(video_path1)
#     cap2 = cv2.VideoCapture(video_path2)

#     # Get video properties
#     fps = int(cap1.get(cv2.CAP_PROP_FPS))
#     width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # Create video writer object
#     fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#     video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#     while True:
#         ret1, frame1 = cap1.read()
#         ret2, frame2 = cap2.read()

#         if not ret1 or not ret2:
#             break

#         # Convert frames to HSV
#         hsv_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)

#         # Define the chroma key color range
#         lower_color = np.array([chroma_color[0] - threshold, chroma_color[1] - threshold, chroma_color[2] - threshold])
#         upper_color = np.array([chroma_color[0] + threshold, chroma_color[1] + threshold, chroma_color[2] + threshold])
#         # lower_color = (36,0,0)
#         # upper_color = (86,255,255)
#         # Create a mask using the chroma key color range
#         mask = cv2.inRange(hsv_frame1, lower_color, upper_color)

#         # Invert the mask
#         inverted_mask = cv2.bitwise_not(mask)

#         # Apply the mask to the original frame
#         foreground = cv2.bitwise_and(frame1, frame1, mask=inverted_mask)

#         # Apply the mask to the second video frame
#         background = cv2.bitwise_and(frame2, frame2, mask=mask)

#         # Combine the foreground and background
#         result = cv2.add(foreground, background)

#         # Write the frame to the output video
#         video_writer.write(result)

#         cv2.imshow("Chroma Keying", result)
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break

#     # Release video capture and writer objects
#     cap1.release()
#     cap2.release()
#     video_writer.release()
#     cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path2 = r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4"  # Replace with the path to your background video
    video_path1 = r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4"  # Replace with the path to your foreground video
    # video_path2 = r"C:\Users\ramak\Downloads\1708635_preview.mp4"
    # video_path1 = r"C:\Users\ramak\Downloads\SampleVideo_1280x720_2mb.mp4"
    output_path = r"C:\Users\ramak\Downloads\chroma_keyed_video.mp4"  # Replace with the desired output path

    chroma_keying(video_path1, video_path2, output_path)


In [50]:
import cv2
import numpy as np

# Load the foreground input video
foreground = cv2.VideoCapture(r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4")

# Load the background input video
background = cv2.VideoCapture(r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4")

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

while True:
    ret1, frame1 = foreground.read()
    ret2, frame2 = background.read()

    if not ret1 or not ret2:
        break

    # Chroma keying
    hsv = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (36, 0, 0), (86, 255,255))
    mask_inv = cv2.bitwise_not(mask)

    # Bitwise-AND mask and original image
    res1 = cv2.bitwise_and(frame1, frame1, mask=mask_inv)
    res2 = cv2.bitwise_and(frame2, frame2, mask=mask)

    # Combine the two results
    final = cv2.add(res1, res2)

    out.write(final)

# Release everything when job is finished
foreground.release()
background.release()
out.release()
cv2.destroyAllWindows()

In [30]:
import cv2
import numpy as np

# Read the input videos
video1 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4")
video2 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4")

# Get the width and height of the videos
width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a new video writer object
video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (width, height))

# Loop over the frames of the videos
while True:

    # Read the next frame from each video
    ret1, frame1 = video1.read()
    ret2, frame2 = video2.read()

    # Check if the frames are empty
    if not ret1 or not ret2:
        break

    # Convert the frames to HSV color space
    hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the green color in HSV color space
    lower_green = np.array([36, 0, 60])
    upper_green = np.array([70, 255, 255])

    # Create a mask for the green color
    mask1 = cv2.inRange(hsv1, lower_green, upper_green)
    # mask2 = cv2.inRange(hsv2, lower_green, upper_green)

    # Apply the mask to the videos
    frame1[mask2 == 1] = (0, 0, 0)
    # frame2[mask1 == 0] = (0, 0, 0)

    # Combine the two videos
    output_frame = cv2.addWeighted(frame1, 0.5, frame2,0.5, 0)

    # Write the output frame to the video file
    video_writer.write(output_frame)

# Release the video capture and writer objects
video1.release()
video2.release()
video_writer.release()

#####$$$$$$$

In [32]:
import cv2
import numpy as np

# Read the input videos
video1 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4")
video2 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4")

# Get the width and height of the videos
width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a new video writer object
video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (width, height))

# Loop over the frames of the videos
while True:

    # Read the next frame from each video
    ret1, frame1 = video1.read()
    ret2, frame2 = video2.read()

    # Check if the frames are empty
    if not ret1 or not ret2:
        break

    # Convert the frames to HSV color space
    hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the green color in HSV color space
    lower_green = np.array([36, 0, 60])
    upper_green = np.array([70, 255, 255])

    # Create a mask for the green color in video 1
    mask1 = cv2.inRange(hsv1, lower_green, upper_green)

    # Set green pixels in video 1 to black
    frame1[mask1 != 0] = (0, 0, 0)

    # Add the pixels from video 2 to the modified frame 1
    # output_frame = cv2.add(frame1, frame2)
    output_frame = cv2.addWeighted(frame1, 0.5, frame2,0.5, 0)


    # Write the output frame to the video file
    video_writer.write(output_frame)

# Release the video capture and writer objects
video1.release()
video2.release()
video_writer.release()


$%%%

In [36]:
import cv2
import numpy as np

# Read the input videos
video1 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\mixkit-switching-to-the-channel-with-a-first-person-control-on-28294-medium.mp4")
video2 = cv2.VideoCapture(r"C:\Users\ramak\Downloads\pattern_-_12313 (720p).mp4")

# Get the width and height of the videos
width = int(video1.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video1.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a new video writer object
video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (width, height))

# Loop over the frames of the videos
while True:

    # Read the next frame from each video
    ret1, frame1 = video1.read()
    ret2, frame2 = video2.read()

    # Check if the frames are empty
    if not ret1 or not ret2:
        break

    # Convert the frames to HSV color space
    hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the green color in HSV color space
    lower_green = np.array([36, 0, 60])
    upper_green = np.array([70, 255, 255])

    # Create a mask for the green color in video 1
    mask1 = cv2.inRange(hsv1, lower_green, upper_green)

    # Invert the mask to select the non-green region
    mask_inv = cv2.bitwise_not(mask1)

    # Extract the green region from video 2
    green_region = cv2.bitwise_and(frame2, frame2, mask=mask1)

    # Extract the non-green region from video 1
    non_green_region = cv2.bitwise_and(frame1, frame1, mask=mask_inv)

    # Combine the green region from video 2 with the non-green region from video 1
    output_frame = cv2.add(green_region, non_green_region)
    # output_frame = cv2.addWeighted(frame1, 0.5, frame2,0.5, 0)


    # Write the output frame to the video file
    video_writer.write(output_frame)

# Release the video capture and writer objects
video1.release()
video2.release()
video_writer.release()
